# 인스타 검색 결과 url 만들어서 접속


In [1]:
from selenium import webdriver
import re
from bs4 import BeautifulSoup 
import time
import pandas as pd

# 인스타 접속 후 로그인하기

In [23]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://www.instagram.com')
time.sleep(2)

In [24]:
email = 'funfun.data0813@gmail.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'dftn7jj7'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.send_keys(password)
input_pw.submit()

time.sleep(2)
close = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
close.click()


### 4-1 인스타 검색 결과 url을 만들어 접속하기

In [25]:
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

### 4-2 selenium으로 url 접속하기

In [26]:
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)

# 첫번째 게시글 열기

### 4-3 html에서 첫번째 게시글 찾아 클릭하기

In [40]:
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

# 게시글 정보 가져오기

### 4-4 게시글 정보 가져오기

In [28]:
def get_content(driver):

    # 현재 페이지 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content=' '
        
    # 본문 내용에서 해시태그 가져오기 (정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]

    # 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like=0
        
    # 위치정보가져오기
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place=''
        
    # 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

# like, place 두개 수집이 안됨

['#속골여름이면 필수인 계곡+백숙 꿀조합!계곡물에 발담그고 먹는 백숙맛 말해모해😆여름이면 여기서 신선놀음 하는거지 ㅇㅈ?@백숙 뿌시러 여기로 가쟈🙋\u200d♀️..#제주맛집 #제주도맛집 #제주맛집추천 #제주핫플 #제주핫플레이스 #제주데이트코스 #제주여행 #제주도여행 #제주도여행코스 #제주여행코스 #제주여행지 #제주여행지추천 #서귀포여행 #제주가볼만한곳 #제주도가볼만한곳',
 '2021-05-10',
 0,
 '',
 ['#속골여름이면',
  '#제주맛집',
  '#제주도맛집',
  '#제주맛집추천',
  '#제주핫플',
  '#제주핫플레이스',
  '#제주데이트코스',
  '#제주여행',
  '#제주도여행',
  '#제주도여행코스',
  '#제주여행코스',
  '#제주여행지',
  '#제주여행지추천',
  '#서귀포여행',
  '#제주가볼만한곳',
  '#제주도가볼만한곳']]

### 4-5 다음 게시글 열기

In [29]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(2)
    
move_next(driver)    

### 4-6 여러 게시글 정보 수집하기

In [42]:
driver = webdriver.Chrome('./chromedriver.exe')

driver.get('https://www.instagram.com')
time.sleep(2)

email = 'funfun.data0813@gmail.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'dftn7jj7'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(3)
close = driver.find_element_by_css_selector('button.sqdOP.L3NKy.y3zKF')
close.click()

time.sleep(3)
close_2 = driver.find_element_by_css_selector('button.aOOlW.bIiDR')
close_2.click()

word = '제주여행'
url = insta_searching(word)

driver.get(url)
time.sleep(3)

select_first(driver)

results = []

target = 50 # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver)
    results.append(data)
    move_next(driver)
    
print(results[:2])

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button.aOOlW.bIiDR"}
  (Session info: chrome=90.0.4430.93)


### 4-7 크롤링 결과 저장하기

In [79]:
results_df =pd.DataFrame(results)
results_df.columns=['content','data','like','place','tags']
results_df.to_excel('D:/Python_Project/portfolio/study_coding/book/데이터 분석 실무 with 파이썬/3_1_crawling_jejuYeoHang.xlsx')

### 4-8 여러 엑셀 파일의 중복을 제거한 후 통합 저장

In [84]:
jeju_insta_df = pd.DataFrame( [ ] )

folder = 'D:/Python_Project/portfolio/study_coding/book/데이터 분석 실무 with 파이썬/'
f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','data','like','place','tags']

### 4- 중복 데이터 제거하고 저장하기

In [87]:
jeju_insta_df.drop_duplicates(subset=['content'], inplace=True)
jeju_insta_df.to_excel('./3_1_crawling_raw.xlsx', index=False)